In [0]:
from fastai.text import *
from data_helpers.Data import *

In [0]:
sent_num_file = ["aspect_0.count", "test_aspect_0.count"]
rating_file = ["aspect_0.rating", "test_aspect_0.rating"]
content_file = ["aspect_0.txt", "test_aspect_0.txt"]

dataset_dir = "./data/hotel_balance_LengthFix1_3000per/"

In [0]:
def concat_to_doc(sent_list, sent_count):
    start_index = 0
    docs = []
    for s in sent_count:
        doc = " . ".join(sent_list[start_index:start_index + s])
        docs.append(doc)
        start_index = start_index + s
    return docs

In [0]:
TRAIN_DATA = 0
TEST_DATA = 1

## load train

In [0]:
# Load Sentence Count --- TRAIN
sent_count_train = list(open(dataset_dir + sent_num_file[TRAIN_DATA], "r").readlines())
sent_count_train = [int(s) for s in sent_count_train if (len(s) > 0 and s != "\n")]
sent_count_train[0:5]

[10, 14, 14, 24, 23]

In [0]:
# Load Labels
aspect_rating_train = list(open(dataset_dir + rating_file[TRAIN_DATA], "r").readlines())
aspect_rating_train = [s for s in aspect_rating_train if (len(s) > 0 and s != "\n")]
print( aspect_rating_train[0:5] )
# Split Labels
aspect_rating_train = [s.split(" ") for s in aspect_rating_train]
aspect_rating_train = np.array(aspect_rating_train)[:, 0:-1]
aspect_rating_train = aspect_rating_train.astype(np.int) - 1
print( aspect_rating_train[0:5] )

['5 5 5 5 5 5 \n', '5 4 5 5 5 5 \n', '2 1 2 2 3 2 \n', '4 3 4 3 5 4 \n', '4 3 2 3 3 5 \n']
[[4 4 4 4 4 4]
 [4 3 4 4 4 4]
 [1 0 1 1 2 1]
 [3 2 3 2 4 3]
 [3 2 1 2 2 4]]


In [0]:
#Load Sentences
sents_train = list(open(dataset_dir + content_file[TRAIN_DATA], "r").readlines())
sents_train = [s.strip() for s in sents_train]
len(sents_train)

265556

In [0]:
sents_train[0:5]

['we had the most wonderful time at the excellence',
 'we opted for the excellence club and it was well worht it for the perks',
 'roberto puello who was the club manager did an extra special job of making us feel welcomed',
 'we took some great pictures that looked like postcards',
 'the food was excellent the pizza was close to being our favorite']

In [0]:
docs_train = concat_to_doc(sents_train, sent_count_train)
docs_train[0]

"we had the most wonderful time at the excellence . we opted for the excellence club and it was well worht it for the perks . roberto puello who was the club manager did an extra special job of making us feel welcomed . we took some great pictures that looked like postcards . the food was excellent the pizza was close to being our favorite . the shows were great especially the michael jackson and the 70's  80's night . the french resturant was probably our favorite . we will definately be back for an anniversary trip . we made some friends but had plenty of privcacy as well . the rooms were great and everytime we left and came back it was cleaned , towels restocked and the bar was as well"

In [0]:
df_train = pd.DataFrame(docs_train)
df_train.head()



,0
0,we had the most wonderful time at the excellen...
1,a truly fantastic place xmas we have just ret...
2,too sick to enjoy the ride to and form the air...
3,"relaxing and fun , but . i rang in the new yea..."
4,somewhat excellence we just returned from a on...


## load test

In [0]:
sent_count_test = list(open(dataset_dir + sent_num_file[TEST_DATA], "r").readlines())
sent_count_test = [int(s) for s in sent_count_test if (len(s) > 0 and s != "\n")]
sent_count_test[0:5]

[9, 30, 25, 33, 29]

In [0]:
aspect_rating_test = list(open(dataset_dir + rating_file[TEST_DATA], "r").readlines())
aspect_rating_test = [s for s in aspect_rating_test if (len(s) > 0 and s != "\n")]
print( aspect_rating_test[0:5] )
aspect_rating_test = [s.split(" ") for s in aspect_rating_test]
aspect_rating_test = np.array(aspect_rating_test)[:, 0:-1]
aspect_rating_test = aspect_rating_test.astype(np.int) - 1
print( aspect_rating_test[0:5] )

['2 1 1 4 2 2 \n', '3 3 2 3 4 4 \n', '5 5 5 4 5 5 \n', '4 3 4 4 4 5 \n', '4 5 4 5 5 5 \n']
[[1 0 0 3 1 1]
 [2 2 1 2 3 3]
 [4 4 4 3 4 4]
 [3 2 3 3 3 4]
 [3 4 3 4 4 4]]


In [0]:
sents_test = list(open(dataset_dir + content_file[TEST_DATA], "r").readlines())
sents_test = [s.strip() for s in sents_test]
len(sents_test)

88444

In [0]:
sents_test[0:5]

["definitely not a 5 star resort i 'm dumbfounded that this hotel gets good reviews and is so highly rated",
 "it 's decidedly a 3 star property , not 5 stars as indicated",
 'the rooms are very dated and run down , old crappy beds and pillows , an old tv and overall poorly maintained',
 'the whole property is pretty run down and old - looking',
 'the food is subpar , not one meal i had would be called great']

In [0]:
docs_test = concat_to_doc(sents_test, sent_count_test)
docs_test[0]

"definitely not a 5 star resort i 'm dumbfounded that this hotel gets good reviews and is so highly rated . it 's decidedly a 3 star property , not 5 stars as indicated . the rooms are very dated and run down , old crappy beds and pillows , an old tv and overall poorly maintained . the whole property is pretty run down and old - looking . the food is subpar , not one meal i had would be called great . the service is uneven and the staff is poorly trained and uninformed . many do not comprehend english . the beach is great , it 's the only redeeming factor . however the resort is a 1- hour taxi trip from the airport"

In [0]:
df_test = pd.DataFrame(docs_test)
df_test.head()


,0
0,definitely not a 5 star resort i 'm dumbfounde...
1,facilities need work . we visited excellence f...
2,excellence was exactly that . my family and i ...
3,"great service , nice hotel , mediocre food . m..."
4,very relaxing experience just returned from my...


### construct LM databunch

In [0]:
lmdb = TextLMDataBunch.from_df("./data/", train_df=df_train, valid_df=df_test, text_cols=0)


In [0]:
lmdb.show_batch()


idx,text
0,"were great especially the michael jackson and the 70 's 80 's night . the french resturant was probably our favorite . we will definately be back for an anniversary trip . we made some friends but had plenty of xxunk as well . the rooms were great and everytime we left and came back it was cleaned , towels restocked and the bar was as well xxbos a truly"
1,"critter outside our room that would not stop chirping . other tips : banana mama 's were great pool drinks , as were coco locos ( drinks served inside coconuts ) . if you 're craving the coconut meat inside the coconut ( which was tasty ) then ask the bartender to crack open the coconut after you ' ve finished your drink . the lobster house was * hot"
2,"spanish language , but willing to make an effort . arrival , departure , and transfers , punta cana airport / excellence : among the easiest we??e ever experienced . breezed through customs on arrival , $ 10 visa fee apiece , porters aggressive but manageable . contacted the resort in advance to arrange a taxi , driver was waiting with a sign . xxunk five minutes , sixty bucks"
3,"horse ride ) will try to swap you sodas and whiskey ( you can requst bottles at no charge for your mini bar ) for some souvineers . they 'll give $ 25 for jack daniels at the ' walmart shack ' . what ever they gave us a price on , we paid half of the half ( ex : asking $ 100 we gave them $ 25 )"
4,"for us . if you??e ever in the usa please look us up ? ? ? . you??e like family to us . great vacation for honeymooner , wedding , anniversaries or if you just want to get away to a quiet and romantic paradise . best part no kids . thanks cecilia garcia xxbos my husband and i are seasoned travellers and we have to say that our eight"


In [0]:
len(lmdb.train_ds.vocab.itos)


23003

In [0]:
x,y = next(iter(lmdb.train_dl))
x[:15,:15].cpu()


tensor([[  104,  2141,   162,    16,    39,    13,  1421,    74,    34,  2680,
            10,   220,    86,    25,    74],
        [   58,    16,    26,    20,  1948,    10,    16,   353,    20,   266,
            13,    83,    98,    11,    79],
        [   27,    17,    15,   211,    16,    91,   253,   241,    64,    10,
           122,  4539,   147,    10,    20],
        [  446,    13,  2037,    19,     9,   615,    10,    21,   966,  4372,
            31,   198,   386,    10,   146],
        [ 2205,    26,     0,    29,  1070,   163,    19,    80,  2112,    10,
            47,   136,   264,    16,    53],
        [   13,   315,   877,   261,    10,    61,    57,  2055,   315,    10,
            37,    15,    41,  2060,   278],
        [  379,    26,  5575,   488,    31,  2148,    12, 18504,    10,   131,
            16,    39,   249,   553,   372],
        [  571,    10,    17,    33,     9,  3962,  3962,    73,    15,   403,
            57,    18,     9,   153,  3717],
        

In [0]:
lmdb.train_ds.vocab.textify(x.cpu()[0,0:10]).split(' ')


['more',
 'holidays',
 'where',
 'we',
 'have',
 'to',
 'worry',
 'about',
 'our',
 'health']

# start train

In [0]:
moms = (0.8,0.7)

In [0]:
learn = language_model_learner(lmdb, AWD_LSTM)
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-2), moms=moms)

KeyboardInterrupt: 